In [13]:
%reload_ext autoreload
%autoreload 2
import pyxdf
import os
import numpy as np
from scipy import stats
from pathlib import Path
import sys
from datetime import datetime
import pandas as pd
import seaborn

from utilities.utl import find_lsl_stream, find_nearest
from utilities.pupil_prep import *
from utilities.SingleSubjectData import SubjectData, Epochs
import importlib


In [9]:
# (import) helper functions
dir_script = Path("__file__").parent.absolute()
dir_rawdata = Path.joinpath(dir_script.parent.absolute(), "04_data","00_raw")
print(dir_script)

c:\Users\User\Desktop\kiel\tremor_feedback_jw\05_scripts


In [10]:
f_list = os.listdir(dir_rawdata)
str_match = 'archer'
fnms = [s for s in f_list if str_match in s]
print([f.split('_')[0] for f in fnms])

['c300', 'c301', 'c302', 'c303', 'c304', 'c305', 'c306', 'c307', 'c308', 'c309', 'c310', 'p200', 'p201', 'p202', 'p203', 'p204', 'p205', 'p206', 'p207', 'p208', 'p209', 'p210', 'p211', 'p212', 'p213', 'p214', 'p215']


In [17]:
sub = SubjectData()
f = fnms[5]
print(f.split('_')[0])
sub.load_data(dir_rawdata,f)

sub.prep_pupil()
    
ev = sub.mrk["time_series"]


c305
Max force is 8067 something
30.00005819136277
30.000053691328503
30.000054291449487
30.000101691344753
30.000054991338402
30.00005869148299
30.000048791407607
30.000055691460148
30.00006609153934
30.000058091478422
30.00005629158113
30.000057791476138


TypeError: Epochs.__init__() missing 1 required positional argument: 'data'

In [58]:
data = np.asanyarray(sub.eye["time_series"],np.float64)
times = np.asanyarray(sub.eye["time_stamps"],np.float64)

events = sub.mrk["time_series"]
type(events)


list

In [44]:
eps = Epochs(sub.eye["time_series"],times=sub.eye["time_stamps"], events=sub.mrk)



AttributeError: Specify events to epoch the data in event_id